# Libraries

In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt


from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *

# Objeto evaluacion

In [2]:
ev = Evaluation()

# Funciones necesarias

## Distancia de Hamming

Calcula la cantidad de genes que se diferencian entre 2 cromosomas, es decir, cuántas posiciones tienen alelos distintos.

In [5]:
padre1 = np.array([1,2,2,5,6,4,7,7,9])
padre2 = np.array([1,2,3,4,3,6,7,8,9])

In [6]:
distancia_hamming(padre1,padre2)

5

## Operador de cruce

Los cruces en CHC se hacen cambiando exactamente la mitad de los elementos que son distintos en ambos padres y luego se aplica Parent Centered o BLX-Alpha

### Cruce Parent Centered

Mutación gaussiana del valor intercambiado.

### Cruce BLX-Alfa

# Experimentación

In [15]:
padre1 = np.array([1,2,2,5,6,4,7,7,9])
padre2 = np.array([1,2,3,4,3,6,7,8,9])

hijos = np.array([[]])

hijos = np.append(hijos, [padre1], 0)
hijos = np.append(hijos, padre2)
hijos

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 9

In [ ]:
def recombinar(poblacion, umbral):

    hijos = None

    for i in range(0,poblacion.shape[0],2):

        padre1 = poblacion[i,:]
        padre2 = poblacion[i+1,:]

        

        if distancia_hamming(padre1, padre2) > umbral:
            
            # swap half the differing bits at random
            hijo = operador_cruce() # crea hijo

            if hijos is None:
                hijos = np.array([hijo])
            else:
                hijos = np.append(hijos,[hijo],0)  

            
    return hijos



In [ ]:
def chc(tama, alpha, ):
    
    t = 0
    d = TOTAL_STATIONS/4

    # Inicializamos poblacion con todos cromosomas generados aleatoriamente
    P = inicializa_poblacion(tama)
    

    # Condicion de parada: numero de rearranques
    while True: 
        t += 1

        # select_r -> hacer un shuffle
        np.random.shuffle(P)

        fitness_p, km_p, slots_p = evalua_poblacion(P,alpha)

        # recombine
        # evaluate
        # select_s






